### Explanation of Library Usage in This Project

In this Jupyter notebook, we leverage several powerful libraries to create an advanced Telegram bot. Each of these libraries plays a crucial role in enhancing the functionality and capabilities of our bot. Below, we provide a comprehensive explanation of each library and the reason for its inclusion.

### Libraries and Their Usage

1. **python-telegram-bot**
   ```python
   !pip install python-telegram-bot==13.7
   ```
   **Purpose:** This library is one of the most popular and powerful tools for building and managing Telegram bots. With it, we can easily connect to the Telegram API to handle messages, commands, and user interactions. We have chosen version 13.7 for its stability and compatibility with our project requirements.

2. **gpt4all**
   ```python
   !pip install gpt4all==2.0.2
   ```
   **Purpose:** The `gpt4all` library allows us to utilize the powerful GPT-4 models for generating intelligent and natural responses. These models provide advanced natural language processing capabilities that significantly enhance the user experience by offering accurate and contextually appropriate answers to user queries.

3. **googletrans**
   ```python
   !pip install googletrans==4.0.0-rc1
   ```
   **Purpose:** `googletrans` is a well-known library for translating text between various languages. Using this library, our bot can translate messages into different languages and provide responses in the user's language, thereby increasing the bot's accessibility and usability.

4. **langid**
   ```python
   !pip install langid
   ```
   **Purpose:** `langid` is a language identification tool that can determine the language of an input text. This capability is critical for our bot to recognize the language of user messages and, if necessary, translate them before generating a response.

5. **transformers**
   ```python
   !pip install transformers
   ```
   **Purpose:** The `transformers` library by Hugging Face offers a wide array of pre-trained language models, including GPT-3 and GPT-4. This library is essential for utilizing advanced NLP models to produce high-quality responses in our bot.

6. **tabulate**
   ```python
   !pip install tabulate
   ```
   **Purpose:** `tabulate` is used for formatting tabular data in a readable way. This library is particularly useful when we need to display structured data or statistics in the bot's responses, ensuring clarity and a professional appearance.

### Conclusion

Utilizing these libraries ensures that our Telegram bot possesses advanced capabilities in natural language processing, translation, language identification, and data management. By integrating these tools, we can create a multilingual and intelligent bot that provides an exceptional user experience. This notebook not only demonstrates the integration of these libraries but also highlights their individual contributions to building a sophisticated conversational agent.

In [1]:
# Install the python-telegram-bot library to interact with the Telegram Bot API
!pip install python-telegram-bot==13.7

# Navigate to the LLM-Telegram-Chatbot directory (this command will only work in a script or shell, not in a Jupyter notebook)
!cd LLM-Telegram-Chatbot

# Clone the LLM-Telegram-Chatbot repository from GitHub
!git clone https://github.com/Fatal3xcept10n/LLM-Telegram-Chatbot.git

# Navigate to the cloned LLM-Telegram-Chatbot directory (again, this command is for shell or script execution)
!cd LLM-Telegram-Chatbot

# Install the gpt4all library, version 2.0.2, to use GPT-4 models for generating responses
!pip install gpt4all==2.0.2

# Install the googletrans library for translation capabilities
!pip install googletrans==4.0.0-rc1

# Install the langid library to identify the language of the text
!pip install langid

# Install the transformers library by Hugging Face for advanced NLP models
!pip install transformers

# Install the tabulate library to format tabular data
!pip install tabulate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.1/490.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.3.3
    Uninstalling cachetools-5.3.3:
      Successfully uninstalled cachetools-5.3.3
/bin/bash: line 1: cd: LLM-Telegram-Chatbot: No such file or directory
Cloning into 'LLM-Telegram-Chatbot'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 58 (delta 26), reused 32 (delta 9), pack-reused 0
Receiving objects: 100% (58/58), 73.09 KiB | 608.00 KiB/s, done.
Resolving deltas: 100% (26/26), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31

In [2]:
# Import necessary modules from the telegram library for handling updates and commands
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters

# Import modules for creating custom keyboard buttons
from telegram import ReplyKeyboardMarkup, KeyboardButton

# Import GPT4All for integrating GPT-4 models
from gpt4all import GPT4All

# Import threading for handling concurrent execution
import threading

# Import googletrans for language translation
from googletrans import Translator

# Import langid for language identification
import langid

# Import logging for logging information and debugging
import logging

# Import sqlite3 for database interactions
import sqlite3

# Import os for operating system interactions
import os

# Import queue for creating and managing queues
import queue


# Set up logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Telegram bot token (replace with your bot token obtained from BotFather)
TOKEN = '7269587692:AAGga__l2lzBjiVFQM9yyxAD-oqjjlRNT_4'

# Select LLM model from gpt4all
model = GPT4All("gpt4all-13b-snoozy-q4_0.gguf")

# Translator instance
translator = Translator()

# Store the contexts and queues for each user
user_contexts = {}
user_queues = {}


# Function to delete and recreate the database
def initialize_database():
    # Check if the database file exists, and remove it if it does
    if os.path.exists('bot_database.db'):
        os.remove('bot_database.db')

    # Connect to (or create) the database
    conn = sqlite3.connect('bot_database.db')
    cursor = conn.cursor()

    # Create the 'users' table with columns: id, user_id, and first_name
    cursor.execute('''
        CREATE TABLE users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER UNIQUE,
            first_name TEXT
        )
    ''')

    # Create the 'questions' table with columns: id, user_id, question, and answer
    cursor.execute('''
        CREATE TABLE questions (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER,
            question TEXT,
            answer TEXT,
            FOREIGN KEY (user_id) REFERENCES users (user_id)
        )
    ''')

    # Commit changes and close the connection
    conn.commit()
    conn.close()

# Initialize the database by calling the function
initialize_database()

# Connect to the database with check_same_thread=False to allow multi-threading
conn = sqlite3.connect('bot_database.db', check_same_thread=False)
cursor = conn.cursor()


# Function to start a conversation for a specific user
def startConversation(chat_id):
    # Use global variables to track user contexts and queues
    global user_contexts, user_queues

    # Create a new chat session with the model
    with model.chat_session():
        while True:
            try:
                # Attempt to get a prompt from the user's queue with a timeout of 5 seconds
                prompt = user_queues[chat_id].get(timeout=5)
                print(f"Working on prompt for user {chat_id}")

                # Generate a response using the prompt
                response = generateResponse(prompt)
                user_contexts[chat_id]['response'] = response

                print(f"Finished processing for user {chat_id}")
                # Set the response event to indicate the response is ready
                user_contexts[chat_id]['response_event'].set()
            except queue.Empty:
                # Exit the loop if the user's context is no longer running
                if chat_id not in user_contexts or not user_contexts[chat_id].get('is_running', False):
                    break
        print(f"Exited conversation loop for user {chat_id}")
# Function to generate a response based on a given prompt
def generateResponse(prompt):
    return model.generate(
        prompt,            # The input prompt for the model
        max_tokens=200,    # Maximum number of tokens to generate
        temp=0.7,          # Sampling temperature for creativity
        top_k=40,          # Limits the sampling pool to the top k tokens
        top_p=0.4,         # Nucleus sampling threshold
        repeat_penalty=1.18, # Penalty for repeating tokens
        repeat_last_n=64,  # Consider last n tokens for repetition penalty
        n_batch=8,         # Number of batches to process in parallel
        n_predict=None,    # Number of tokens to predict (None for default)
        streaming=False    # Disable streaming for generating response
    )

# Function to handle the /start command and initialize conversation
def start(update, context):
    # Get the chat ID and user's first name from the update message
    chat_id = update.message.chat_id
    user_first_name = update.message.from_user.first_name
    context.user_data['first_name'] = user_first_name

    # Initialize user context and set running status
    if chat_id not in user_contexts:
        user_contexts[chat_id] = {}
    user_contexts[chat_id]['is_running'] = True
    user_contexts[chat_id]['response_event'] = threading.Event()

    # Initialize user queue if not already present
    if chat_id not in user_queues:
        user_queues[chat_id] = queue.Queue()

    # Insert or update user information in the database
    cursor.execute("INSERT OR IGNORE INTO users (user_id, first_name) VALUES (?, ?)", (chat_id, user_first_name))
    cursor.execute("UPDATE users SET first_name = ? WHERE user_id = ?", (user_first_name, chat_id))
    conn.commit()

    # Start a new thread for the conversation
    conversation_thread = threading.Thread(target=startConversation, args=(chat_id,))
    conversation_thread.start()

    # Send a welcome message to the user with a custom keyboard
    context.bot.send_message(chat_id=chat_id,
                             text=f"Hello {user_first_name}! Welcome to the Ask me! bot. If you have any questions, please press the Ask me! button.",
                             reply_markup=main_menu_keyboard())

# Function to handle the /help command and provide usage instructions
def help_command(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text='To use this bot, you can choose from the following options:\n'
                                  '1. Start: Begin interacting with the bot.\n'
                                  '2. Ask me!: Ask any question you have.\n'
                                  'Simply type your message or select an option from the menu.')

# Function to handle the Ask me! command and prompt the user to ask a question
def ask_me(update, context):
    chat_id = update.message.chat_id
    user_first_name = context.user_data.get('first_name', 'User')
    context.bot.send_message(chat_id=chat_id, text=f"{user_first_name}, please type your question below:")

# Function to handle exiting the bot and clear user data
def exit_bot(update, context):
    chat_id = update.message.chat_id
    user_first_name = context.user_data.get('first_name', 'User')
    user_username = update.message.from_user.username

    # Determine the name to use in the exit message
    if user_username:
        user_name = user_username
    else:
        user_name = user_first_name

    # Send a thank you message to the user
    context.bot.send_message(chat_id=chat_id, text=f"Thank you {user_name} for using the bot. We hope to assist you again soon!")

    # Clear user-specific data
    context.user_data.clear()
    context.chat_data.clear()

    # Update user context to indicate the conversation has ended
    if chat_id in user_contexts:
        user_contexts[chat_id]['is_running'] = False
        del user_contexts[chat_id]
    if chat_id in user_queues:
        del user_queues[chat_id]

# Function to handle incoming messages and process responses
def handleMessage(update, context):
    chat_id = update.message.chat_id
    user_message = update.message.text
    print(f"Received message from user: {user_message}")

    # Initialize user queue and context if not already present
    if chat_id not in user_queues:
        user_queues[chat_id] = queue.Queue()
    if chat_id not in user_contexts:
        user_contexts[chat_id] = {
            'is_running': True,
            'response_event': threading.Event(),
        }

    # Detect the language of the user message
    lang, _ = langid.classify(user_message)

    # Translate message to English if it is in Persian
    if lang == 'fa':
        translated_prompt = translator.translate(user_message, src='fa', dest='en').text
    else:
        translated_prompt = user_message

    # Add the translated prompt to the user's queue
    user_queues[chat_id].put(translated_prompt)

    # Wait for the response to be ready
    user_contexts[chat_id]['response_event'].wait()
    user_contexts[chat_id]['response_event'].clear()

    # Retrieve the generated response
    response = user_contexts[chat_id]['response']

    # Translate the response back to Persian if the original message was in Persian
    if lang == 'fa':
        translated_response = translator.translate(response, src='en', dest='fa').text
    else:
        translated_response = response

    # Insert the question and answer into the database
    cursor.execute("INSERT INTO questions (user_id, question, answer) VALUES (?, ?, ?)", (chat_id, user_message, translated_response))
    conn.commit()

    # Send the translated response back to the user with a custom keyboard
    context.bot.send_message(chat_id=chat_id,
                             text=translated_response,
                             reply_markup=main_menu_keyboard())

# Function to handle button presses and delegate to appropriate handlers
def handle_button(update, context):
    try:
        chat_id = update.effective_chat.id
        text = update.message.text

        # Check the text of the button pressed and call the corresponding function
        if text == 'Start':
            start(update, context)
        elif text == 'Ask me!':
            ask_me(update, context)
        elif text == 'Help':
            help_command(update, context)
        elif text == 'Exit':
            exit_bot(update, context)
        else:
            handleMessage(update, context)
    except Exception as e:
        logger.error(f"Error in handle_button: {e}")

# Function to create main menu keyboard
def main_menu_keyboard():
    keyboard = [
        [KeyboardButton('Start'), KeyboardButton('Ask me!'), KeyboardButton('Help')],
        [KeyboardButton('Exit')]
    ]
    return ReplyKeyboardMarkup(keyboard, resize_keyboard=True, one_time_keyboard=False)

# Main function to set up the bot and start polling for updates
def main():
    updater = Updater(token=TOKEN, use_context=True)
    dispatcher = updater.dispatcher

    # Handlers for different commands and messages
    start_handler = CommandHandler('start', start)
    help_handler = CommandHandler('help', help_command)
    button_handler = MessageHandler(Filters.text & (~Filters.command), handle_button)
    message_handler = MessageHandler(Filters.text & (~Filters.command), handleMessage)

    # Adding handlers to the dispatcher
    dispatcher.add_handler(start_handler)
    dispatcher.add_handler(help_handler)
    dispatcher.add_handler(button_handler)  # Add button_handler
    dispatcher.add_handler(message_handler)

    # Start polling for updates
    updater.start_polling()
    updater.idle()

    # Close the database connection
    conn.close()

if __name__ == '__main__':
    main()
